# A2: Enhanced Linear Regression - Improved Pipeline
**Student ID: st126010 - Htut Ko Ko**

This notebook implements enhanced linear regression with polynomial features and Lasso regularization using proper data pipeline: split > impute > scale > polynomial > train

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso as SklearnLasso
from LinearRegression import LinearRegression, Ridge 
import mlflow
import pickle
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

In [ ]:
# Load the dataset
data = pd.read_csv('Cars.csv')
print(f"Dataset shape: {data.shape}")
print(f"Columns: {list(data.columns)}")
data.head()

In [ ]:
# Data info and missing values
print("Dataset Info:")
data.info()
print("\nMissing values before cleaning:")
print(data.isnull().sum())

In [ ]:
# Clean data - extract numeric values from string columns
data['mileage'] = data['mileage'].str.extract('(\\d+\\.?\\d*)').astype(float)
data['engine'] = data['engine'].str.extract('(\\d+)').astype(float)
data['max_power'] = data['max_power'].str.extract('(\\d+\\.?\\d*)').astype(float)

# Drop irrelevant columns
data = data.drop(columns=['name', 'torque'])

print("Data cleaning completed")
print(f"Shape after cleaning: {data.shape}")
print("\nMissing values after cleaning:")
print(data.isnull().sum())

In [ ]:
# Define features and target
numeric_columns = ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'seats']
categorical_columns = ['fuel', 'seller_type', 'transmission', 'owner']
feature_names = numeric_columns + categorical_columns

X = data[feature_names].copy()
y = data['selling_price']

print(f"Features: {feature_names}")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Missing values in features:")
print(X.isnull().sum())

In [ ]:
# STEP 1: Split data FIRST (before any preprocessing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Training missing values: {X_train.isnull().sum().sum()}")
print(f"Test missing values: {X_test.isnull().sum().sum()}")

In [ ]:
# STEP 2: Impute missing values (fit on train, transform both)
imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

# Impute numeric columns
X_train[numeric_columns] = imputer_num.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = imputer_num.transform(X_test[numeric_columns])

# Impute categorical columns
X_train[categorical_columns] = imputer_cat.fit_transform(X_train[categorical_columns])
X_test[categorical_columns] = imputer_cat.transform(X_test[categorical_columns])

print("✅ Missing values imputed")
print(f"Training missing values after imputation: {X_train.isnull().sum().sum()}")
print(f"Test missing values after imputation: {X_test.isnull().sum().sum()}")

In [ ]:
# STEP 3: Encode categorical variables (fit on train, transform both)
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    label_encoders[col] = le

print("✅ Categorical variables encoded")
print(f"Label encoders created for: {list(label_encoders.keys())}")

In [ ]:
# STEP 4: Scale features (fit on train, transform both)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Features scaled")
print(f"Scaled training set shape: {X_train_scaled.shape}")
print(f"Scaled test set shape: {X_test_scaled.shape}")

In [ ]:
# STEP 5: Create polynomial features (fit on train, transform both)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

print("✅ Polynomial features created")
print(f"Polynomial training set shape: {X_train_poly.shape}")
print(f"Polynomial test set shape: {X_test_poly.shape}")

In [ ]:
# Cross-validation function
def cross_validate_model(model, X_data, y_data, cv_folds=5):
    """
    Perform cross-validation for a given instantiated model.
    """
    kfold = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    r2_scores = []
    mse_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_data)):
        # Split data for this fold
        X_fold_train, X_fold_val = X_data[train_idx], X_data[val_idx]
        y_fold_train, y_fold_val = y_data.iloc[train_idx], y_data.iloc[val_idx]
        
        # Train model
        model.fit(X_fold_train, y_fold_val.values)
        
        # Predict and evaluate
        y_pred = model.predict(X_fold_val)
        r2 = model.r2(X_fold_val, y_fold_val.values)
        mse = mean_squared_error(y_fold_val.values, y_pred)
        
        r2_scores.append(r2)
        mse_scores.append(mse)
    
    return {
        'mean_r2': np.mean(r2_scores),
        'std_r2': np.std(r2_scores),
        'mean_mse': np.mean(mse_scores),
        'std_mse': np.std(mse_scores),
    }

print("Cross-validation function defined!")

In [ ]:
# Define experiment configurations
experiments = {
    'linear_polynomial': {
        'model': LinearRegression(),
        'data': X_train_poly,
        'description': 'Linear Regression with Polynomial Features'
    },
    'ridge_polynomial': {
        'model': Ridge(alpha=0.1),
        'data': X_train_poly,
        'description': 'Ridge Regression with Polynomial Features'
    },
    'lasso_polynomial': {
        'model': SklearnLasso(alpha=0.1, random_state=42),
        'data': X_train_poly,
        'description': 'Lasso Regression with Polynomial Features'
    }
}

print("Experiment configurations defined!")

In [ ]:
# Run experiments with cross-validation
results = []
best_model = None
best_score = -np.inf

print("Running experiments with cross-validation...\n")

for exp_name, config in experiments.items():
    print(f"Running {exp_name}: {config['description']}")
    
    # Perform cross-validation
    cv_results = cross_validate_model(config['model'], config['data'], y_train)
    
    # Store results
    result = {
        'experiment_name': exp_name,
        'model_type': type(config['model']).__name__,
        'mean_r2': cv_results['mean_r2'],
        'std_r2': cv_results['std_r2'],
        'mean_mse': cv_results['mean_mse'],
        'std_mse': cv_results['std_mse']
    }
    results.append(result)
    
    # Track best model
    if cv_results['mean_r2'] > best_score:
        best_score = cv_results['mean_r2']
        best_model = config
        best_model['name'] = exp_name
    
    print(f"  Mean R²: {cv_results['mean_r2']:.4f} (±{cv_results['std_r2']:.4f})")
    print(f"  Mean MSE: {cv_results['mean_mse']:.2f} (±{cv_results['std_mse']:.2f})\n")

print(f"Best model: {best_model['name']} with R² = {best_score:.4f}")

In [ ]:
# Train and evaluate the best model on the test set
if best_model:
    print("Training Best Model on Full Dataset")
    print(f"Best Configuration: {best_model['name']}")
    
    # Train on full training set
    final_model = best_model['model']
    final_model.fit(best_model['data'], y_train.values)
    
    # Predict on test set
    if best_model['name'] == 'lasso_polynomial':
        y_pred_test = final_model.predict(X_test_poly)
        test_r2 = final_model.r2(X_test_poly, y_test.values)
    else:
        y_pred_test = final_model.predict(X_test_poly)
        test_r2 = r2_score(y_test, y_pred_test)
    
    test_mse = mean_squared_error(y_test, y_pred_test)
    
    print(f"\nFinal Test Results:")
    print(f"- Test R2 Score: {test_r2:.4f}")
    print(f"- Test MSE: {test_mse:.2f}")
    
    # Save the model and all necessary artifacts
    model_artifacts = {
        'model': final_model,
        'scaler': scaler,
        'poly': poly,
        'imputer_num': imputer_num,
        'imputer_cat': imputer_cat,
        'label_encoders': label_encoders,
        'feature_names': feature_names,
        'metrics': {
            'test_r2': test_r2,
            'test_mse': test_mse,
            'cv_mean_r2': best_score
        },
        'model_type': best_model['name']
    }
    
    with open('a2_model_artifacts.pkl', 'wb') as f:
        pickle.dump(model_artifacts, f)
    
    print("\nModel saved as 'a2_model_artifacts.pkl'")
    print(f"🎯 Final Test R²: {test_r2:.4f}")
    print("🔄 Pipeline: Load → Clean → Split → Impute → Scale → Polynomial → Train → Evaluate")
else:
    print("No experiments were run successfully.")

In [ ]:
# Visualize results
if best_model and 'y_pred_test' in locals():
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.scatter(y_test, y_pred_test, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Actual Price')
    plt.ylabel('Predicted Price')
    plt.title(f'A2: Actual vs Predicted (R² = {test_r2:.4f})')
    
    plt.subplot(1, 2, 2)
    residuals = y_test - y_pred_test
    plt.scatter(y_pred_test, residuals, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Price')
    plt.ylabel('Residuals')
    plt.title('Residual Plot')
    
    plt.tight_layout()
    plt.show()
    
    # Display results summary
    results_df = pd.DataFrame(results)
    print("\nExperiment Results Summary:")
    print(results_df[['experiment_name', 'model_type', 'mean_r2', 'std_r2']].round(4))